In [7]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5
%connections glue_delta_connection
%additional_python_modules delta-spark
%extra_py_files s3://my-python-demo-jars/delta-core_2.12-2.1.0.jar,s3://my-python-demo-jars/delta-spark_2.13-3.1.0.jar,s3://my-python-demo-jars/delta-storage-3.1.0.jar

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


...

Connections to be included:
glue_delta_connection


You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
delta-spark


You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Extra py files to be included:
s3://my-python-demo-jars/delta-core_2.12-2.1.0.jar
s3://my-python-demo-jars/delta-spark_2.13-3.1.0.jar
s3://my-python-demo-jars/delta-storage-3.1.0.jar



In [18]:
%%configure
{
     "conf": {         
        "spark.jars.packages": "io.delta:delta-core_2.12-2.1.0.jar",
        "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog",
         "spark.delta.logStore.class":"org.apache.spark.sql.delta.storage.S3SingleDriverLogStore"
    },
    "datalake-formats":"delta"
}

You are already connected to a glueetl session 1c738945-de6d-4464-bed7-ae132f8a22a9.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'conf': {'spark.jars.packages': 'io.delta:delta-core_2.12-2.1.0.jar', 'spark.sql.extensions': 'io.delta.sql.DeltaSparkSessionExtension', 'spark.sql.catalog.spark_catalog': 'org.apache.spark.sql.delta.catalog.DeltaCatalog', 'spark.delta.logStore.class': 'org.apache.spark.sql.delta.storage.S3SingleDriverLogStore'}, 'datalake-formats': 'delta'}


In [8]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from awsglue.dynamicframe import DynamicFrame
import boto3
from delta import *


def parse_s3_path(path):
    parts = path.split('/')
    bucket_name = parts[2]
    prefix = '/'.join(parts[3:])
    return bucket_name, prefix


def s3_object_cnt(path):

    s3_client = boto3.client('s3')
    bucket_name, prefix = parse_s3_path(path)
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if len(response['Contents']) < 2 and response['Contents'][0]['Size'] == 0:
        cnt = 0
        print("count is",cnt)
    else:
        cnt = 1
        
    print(f'Object count in {bucket_name} and {prefix} prefix is {cnt}')
    return cnt


def add_surrogate_key(df: DataFrame, name: str):
    """
    This function generates a surrogate key using a combination of column values and current timestamp.
    """
    timestamp_column = current_timestamp()    
    unique_key_column = lit(str(uuid.uuid4()))
    surrogate_key_column = sha2(concat_ws("_", *df.columns, timestamp_column, unique_key_column), 256)

    


def create_delta_tbl(df, path):  
    
    df.write.format("delta").mode("append").save(path)
    
def create_stage_delta_tbl(df, path):
    """
    This function will overwrites exsiting data in s3 prefix and write dataframe to S3 in deltalake format.
    """
    df.write.format("delta").mode("overwrite").option(
        "mergeSchema", "true").save(path)
    return DeltaTable.forPath(spark, path)       


In [9]:
from pyspark.sql.window import Window
def employee_capture_changes(stage_tbl, base_tbl):
    """
    This function will identify chnages between source data and target deltalake dataset.

    """
    updates_df = stage_tbl.toDF().alias("stage")\
        .join(base_tbl.alias("base"), col("stage.customer_id") == col("base.customer_id"), "left")\
        .filter((col('base.Customer_SurrogateKey').isNull()) | (col('base.Customer_SurrogateKey') != col('stage.Customer_SurrogateKey')))
    

    cols = ['customer_id', 'first_name', 'last_name', 'city',
            'phone', 'state', 'Customer_SurrogateKey','zip','customer_class','email','delete_flag']
    stage_updates = updates_df.select(
        'stage.*').withColumn("delete_flag", lit(False)).select(*cols)
    base_updates = updates_df.select('base.*').drop('isCurrent', 'start_date', 'end_date').select(*cols)

    # perform union
    union_updates = base_updates.union(
        stage_updates).filter(col('Customer_SurrogateKey').isNotNull())

    # identify deleted records
    base_tbl.alias('base').join(
        stage_tbl.toDF().alias('stage'), 'customer_id', 'anti')

    drop_del_cols = ['delete_flag', 'start_date', 'end_date', 'isCurrent']
    del_df = base_tbl.alias('base')\
        .join(stage_tbl.toDF().alias('stage'), 'customer_id', 'anti')\
        .drop(*drop_del_cols)\
        .withColumn("merge_delete_flag", lit(True))\
        .withColumnRenamed('merge_delete_flag', 'delete_flag')

    union_updates_dels = union_updates.union(del_df)

    return union_updates_dels


def employee_delta_records(base_tbl, union_updates_dels):
    """
    This function will identify delete records and sets delete_flag to true and updates end_date.
    """
    
    delete_join_cond = "customer.customer_id=customerUpdates.customer_id and customer.Customer_SurrogateKey = customerUpdates.Customer_SurrogateKey"
    delete_cond = "customer.Customer_SurrogateKey == customerUpdates.Customer_SurrogateKey and customer.isCurrent = true and customerUpdates.delete_flag = true"

    base_tbl.alias("customer").merge(union_updates_dels.alias("customerUpdates"), delete_join_cond).whenMatchedUpdate(
        condition=delete_cond, set={"isCurrent": "false", "end_date": current_date(), "delete_flag": "true"}).execute()


def employee_upsert_records(base_tbl, union_updates_dels):
    """
    This function will identify insert/update records and performs merge operation on exsiting deltalake dataset.
    """
    upsert_cond = "customer.customer_id=customerUpdates.customer_id and customer.Customer_SurrogateKey = customerUpdates.Customer_SurrogateKey and customer.isCurrent = true"
    upsert_update_cond = "customer.isCurrent = true and customerUpdates.delete_flag = false"

    base_tbl.alias("customer").merge(union_updates_dels.alias("customerUpdates"), upsert_cond).whenMatchedUpdate(condition=upsert_update_cond, set={"isCurrent": "false", "end_date": current_date()}).whenNotMatchedInsert(
        values={"isCurrent": "true", "customer_id": "customerUpdates.customer_id", "first_name": "customerUpdates.first_name", "last_name": "customerUpdates.last_name", "city": "customerUpdates.city", "phone": "customerUpdates.phone",
                "state": "customerUpdates.state", "Customer_SurrogateKey": "customerUpdates.Customer_SurrogateKey","zip":"customerUpdates.zip","customer_class":"customerUpdates.customer_class","email":"customerUpdates.email", "start_date": current_date(), "delete_flag":  "customerUpdates.delete_flag", "end_date": "null"}).execute()


def employees_scd(stage_s3_path, processed_s3_path):
    """
    This function will perform scd type 2 on deltalake dataset on s3 for employees sample dataset. 
    """
    srcDyf =  glueContext.create_dynamic_frame.from_catalog(database="data_lake_catalog_db", table_name="silver_customers")    
        
    srcDF = create_Surrogatekey(srcDyf.toDF(), 'Customer_SurrogateKey')
    
    df2 = srcDF.withColumn("rn", row_number().over(Window.partitionBy("Customer_SurrogateKey").orderBy("customer_id")))
    df3 = df2.filter("rn = 1").drop("rn")

    if s3_object_cnt(processed_s3_path) != 0:
        print(f'Deltalake customer data already exists. started loading data ...')

        # delta lake satge table for input data source.
        stage_tbl = create_stage_delta_tbl(df3, stage_s3_path)

        # read delta lake base table for current records
        base_tbl = DeltaTable.forPath(spark, processed_s3_path)

        base_tbl_df = base_tbl.toDF().where((col('isCurrent') == "true")
                                            & (col('delete_flag') == "false"))

        union_updates_dels = employee_capture_changes(stage_tbl, base_tbl_df)

        # perform soft deletes
        employee_delta_records(base_tbl, union_updates_dels)

        # refresh base table and perform merge operation
        base_tbl = DeltaTable.forPath(spark, processed_s3_path)

        employee_upsert_records(base_tbl, union_updates_dels)

    else:
        print(f'Delta lake customer data dosent exists. Performing inital data load ...')

        srcDF = srcDF.withColumn("start_date", current_date()).withColumn("end_date", lit(None).cast(
            StringType())).withColumn("isCurrent", lit(True)).withColumn("delete_flag", lit(False))

        create_delta_tbl(srcDF, processed_s3_path)

stage_s3_path="s3://data-lake-demo-stage/customer/"
processed_s3_path="s3://data-lake-demo-medallion/gold/customers/"
employees_scd(stage_s3_path, processed_s3_path)
   

Object count in data-lake-demo-medallion and gold/customers/ prefix is 1
Deltalake customer data already exists. started loading data ...
